In [17]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [18]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [19]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        if (len(self.results_df) < 5):
            return
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self, metrics1, metrics2):
        if (len(self.results_df) < 5):
            return
        r2_val, r2_test = self.results_df[metrics1], self.results_df[metrics2]
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation(metrics1="r2", metrics2="r2_vt")
        self.discard_high_standard_deviation(metrics1="r2_val", metrics2="r2_test")
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [20]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_8_6_1,0.925631,0.771848,0.547149,0.681218,0.013913,0.025429,0.039963,0.032268,"Hidden Size=[35, 17], regularizer=0.1, learnin..."
2,model_8_6_0,0.923885,0.814241,0.538366,0.702326,0.014239,0.020704,0.040738,0.030132,"Hidden Size=[35, 17], regularizer=0.1, learnin..."
52,model_8_7_0,0.889850,0.542025,0.684965,0.638856,0.020607,0.028626,0.043384,0.035571,"Hidden Size=[35, 17], regularizer=0.1, learnin..."
72,model_10_5_0,0.872568,0.924062,0.632562,0.833128,0.023840,0.023851,0.059256,0.040513,"Hidden Size=[45, 22], regularizer=0.1, learnin..."
77,model_10_5_1,0.871554,0.905705,0.574788,0.802495,0.024029,0.029617,0.068573,0.047949,"Hidden Size=[45, 22], regularizer=0.1, learnin..."
83,model_5_7_1,0.870279,0.707323,0.776090,0.791958,0.024268,0.032432,0.030639,0.031588,"Hidden Size=[30, 15], regularizer=0.01, learni..."
86,model_10_5_2,0.869267,0.883968,0.529362,0.773407,0.024457,0.036444,0.075899,0.055011,"Hidden Size=[45, 22], regularizer=0.1, learnin..."
